In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
sys.path.insert(0, '../')
from humannotator.humannotator import Annotator
from humannotator.data import Data_DataFrame, Data
from humannotator.annotations import Task_MultipleChoice, Annotations
import pandas as pd

In [3]:
df = pd.read_csv('news.csv', index_col=0)
data = Data_DataFrame(df, element_col='title', id_col='news_id')

In [4]:
instruction = "Choose one of the following options:"
choices={
    '0': 'not adverse media',
    '1': 'adverse media',
    '3': 'exclude from dataset'
    }

task = Task_MultipleChoice(instruction=instruction, choices=choices)
annotations = Annotations(task)
annotator = Annotator(data, annotations)

In [ ]:
annotator(data.ids)

id: 052632_2015-02-28
element:
    Rand Paul wins 2015 CPAC straw poll
Choose one of the following options:
[0] - not adverse media
[1] - adverse media
[3] - exclude from dataset
[.] - exit


In [6]:
annotator.annotations.annotations

{0: Annotation(value: 0, timestamp: 2019-08-06 00:51:03.378904),
 1: Annotation(value: 3, timestamp: 2019-08-06 00:51:04.808898),
 2: Annotation(value: 1, timestamp: 2019-08-06 00:51:06.189198),
 3: Annotation(value: 0, timestamp: 2019-08-06 00:51:10.866436)}